# Hugging Face Sentiment Analysis Deployment
In this notebook I import a Twitter-specific sentiment anaysis model hosted on Hugging Face's pre-trained model hub and I deploy it as an SageMaker endpoint.

## Table of contents: <a class="anchor" id="home"></a>

* [1. Set up enviornment](#env)
  * [1.1 Install packages](#packages)
  * [1.2 Import modules](#modules)
  * [1.3 Initialize session objects](#sess)
* [2. Import model](#import)
  * [2.1 Download model](#download)
* [3 Deploy model](#deploy)

## 1. Set up environment <a class="anchor" id="env"></a>
 [Back to top](#home)

### 1.1 Install packages <a class="anchor" id="packages"></a>
 [Back to top](#home)

In [3]:
# INSTALL packages if not installed
!pip install -U transformers
!pip install -U sagemaker

### 1.2 Import modules <a class="anchor" id="modules"></a>
 [Back to top](#home)

In [4]:
from transformers import pipeline
import pandas as pd
import os
import json

# AWS
import boto3
# AWS SageMaker
import sagemaker
from sagemaker.huggingface import HuggingFaceModel
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.multidatamodel import MultiDataModel
from sagemaker.predictor import Predictor

### 1.3 Initialize session objects <a class="anchor" id="sess"></a>
 [Back to top](#home)

In [6]:
# Set up SageMaker session object
sess = sagemaker.Session()


# Set up sagemaker session bucket. This is used for uploading data, models, and logs
sagemaker_session_bucket = None
# Set up role
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName = 'sagemaker_execution_role')['Role']['Arn']

if sagemaker_session_bucket is None and sess is not None:
    # Set to default bucket if no bucket name given
    sagemaker_session_bucket = sess.default_bucket()

# specify bucket location for session
sess = sagemaker.Session(default_bucket = sagemaker_session_bucket)

# Specify region for session
region = sess.boto_region_name

# Initialize sagemaker client
sm_client = boto3.client('sagemaker')


print(f'sagemaker role arn: {role}')
print(f'sagemaker bucket: {sess.default_bucket()}')
print(f'sagemaker session region: {region}')

sagemaker role arn: arn:aws:iam::034604206629:role/service-role/AmazonSageMaker-ExecutionRole-20220520T212632
sagemaker bucket: sagemaker-ca-central-1-034604206629
sagemaker session region: ca-central-1


## 2. Import model <a class="anchor" id="import"></a>
 [Back to top](#home)

### 2.1 Download model <a class="anchor" id="download"></a>
 [Back to top](#home)

In [7]:
# CREATE model path
model_id = r'cardiffnlp/twitter-roberta-base-sentiment-latest'

# CREATE payload object to Hugging Face hub
hub = {'HF_MODEL_ID': model_id,
         'HF_TASK': 'text-classification'}

# CREATE URI object to kubernetes environment to deploy
URI = '763104351884.dkr.ecr.ca-central-1.amazonaws.com/huggingface-pytorch-inference:1.10.2-transformers4.17.0-cpu-py38-ubuntu20.04'

# INITIALIZE transformer object
model = HuggingFaceModel(
                env=hub,
                role=role,
                image_uri=URI)

## 3. Deploy model <a class="anchor" id="deploy"></a>
 [Back to top](#home)

In [9]:
# DEPLOY to SageMaker endpoint
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    endpoint_name='twitter-sentiment-analyzer',
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer())

-----!